<a href="https://colab.research.google.com/github/uwcmg/RNAseq/blob/main/udn_ucla_external_uwcmg_fam287604_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>RNA-seq workshop @ ASHG</h1>

A hands-on rare disease diagnostic workshop on RNA-seq data [#ASHG20](https://twitter.com/hashtag/ASHG20)

The accompanying presentation can be found here: [tinyurl.com/RNA-ASHG-presentation](https://tinyurl.com/RNA-ASHG-presentation).

And if any troubles arise please refer to the FAQ [here](https://tinyurl.com/RNA-ASHG-FAQ).

For any further discussion please sign up at our [Slack community](https://join.slack.com/t/rnaseq4rare/shared_invite/enQtNzU1MzEyOTI4NjQ3LTMwNjM1NmZlNWY3Nzk1MjkxYTFkYjBjNjdlNTI3Y2ZkNzJjZTNmZTFiZDVhOTVhMjEwYjRiYzA4Y2QwMzhjNTA
).

---

# Setup the notebook

To get started, we have to setup this IR notebook @ Colab. To do so, please run the following code snippet.
This will download all necessary software packages for this tutorial. Also, it will download the example datasets used throughout this tutorial.

This part has to be run each time the virtual machine stops. Usually after 60 minutes idle time.

In [1]:
# takes ~3 minutes
download.file(destfile="r-env-setup-script.R", 
    url="https://raw.githubusercontent.com/c-mertes/RNAseq-ASHG19/master/r-env-setup-script.R")
source("r-env-setup-script.R")
print("Setup done.")

Update and install needed Ubuntu packages

Download R package cache

Unzipping R package cache

Retrieve data for tutorials



[1] "Setup done."


If you do see an error or warning in the setup, please reset the notebook and try it again:

`Runtime -> Factory reset runtime`

# Expression Outlier Detection in RNA-seq data

A short introduction to expression outlier detection can be found [here](https://docs.google.com/presentation/d/1a7KZ6FXwVmGqF-FMAnz0QTk07WgzRWAbFhSNrUBspIQ/edit#slide=id.g630401e960_5_0).





## Data import
---

First, we have to load the [*OUTRIDER*](http://bioconductor.org/packages/release/bioc/html/OUTRIDER.html) package that we will use to run the full expression outlier analysis. To map ENSEMBL gene names and HGNC symbols we use [*annotables*](https://github.com/stephenturner/annotables). For efficient data handling and visualization, we further load [*data.table*](https://cran.r-project.org/web/packages/data.table/index.html), [*ggplot2*](https://cran.r-project.org/web/packages/ggplot2/index.html), and [*ggpubr*](https://cran.r-project.org/web/packages/ggpubr/index.html).

In [ ]:
library(OUTRIDER)
library(annotables)
library(data.table)
library(ggplot2)
library(ggpubr)

For the gene expression outlier detection analysis, we need the raw read counts and a sample annotation. Let's load them into the session.

Have a look at the annotations we use and how many genes were counted.

In [ ]:
# sample annotation and raw read counts
anno <- fread("./annotation.tsv")[, 1:6]
cts  <- as.matrix(read.table("./outrider/raw_counts.tsv.gz"))

# look into the data
head(anno)
print("Dimensions of the annotation:")
dim(anno)

cts[1:5, 1:10]
print("Dimensions of the count table:")
dim(cts)

We can see that we have ~60,000 genes and 100 samples in our experiment. For counting, we used all the genes annotated in the [GENCODE v29 annotation](https://www.gencodegenes.org/human/release_29lift37.html).

Create an *OUTRIDER* object with the loaded annotation and raw count matrix by running `OutriderDataSet`.

In [ ]:
anno[,sampleID:=INDIVIDUAL]
ods <- OutriderDataSet(countData=cts, colData=anno)
ods

## Quality control and preprocessing of raw count data

In this tutorial, only a few quality control metrics are explored. In a real experiment setup, this step should be done more extensively ([see slides for hints](https://docs.google.com/presentation/d/1a7KZ6FXwVmGqF-FMAnz0QTk07WgzRWAbFhSNrUBspIQ/edit?pli=1#slide=id.g64b01c344e_4_51)). 

### Size factor

One important metric is the *sizeFactor*. The *sizeFactor* represents the sequencing depth of each sample with respect to the others and is centered around 1. The *sizeFactor* can be estimated with `estimateSizeFactors` based on the [*DESeq2*](http://bioconductor.org/packages/release/bioc/html/DESeq2.html) function. In gene expression analysis, one of the first normalization steps is done using the *sizeFactor*.

In [ ]:
# estimate sizeFactors 
ods <- estimateSizeFactors(ods)

# visualize the sizeFactors
plotSizeFactors(ods)

As expected, most data points are around 1. A low *sizeFactor* could be an indication for a failed experiment or other issues during the preparation (eg. low RNA input). Those samples should be dealt with caution. Also high *sizeFactors* should be investigated. 

Let's have a look at the samples with the lowest *sizeFactor*. 

In [ ]:
round(sort(sizeFactors(ods)), digits=2)[1:5]

### Filter non-expressed genes

Not all genes are expressed in a given tissue/sample. It depends on the tissues and the sequencing depth how many expressed genes are expected. 

To make the modelling and outlier detection more robust, a good filtering of non-expressed genes is crucial. The cutoffs are dependent on the project and should be defined beforehand. The FPKM value (fragments per kilobase of million mapped reads) is a good measurement for this.

Here, we keep genes where at least 5% of the samples have an FPKM value greater than 1.

To compute the FPKM value, we need the gene length from the annotation file. 

In [ ]:
# load gene annotation to compute FPKM values
txdb <- loadDb("annotations/gencode.v29lift37.annotation.txdb")
ods <- filterExpression(ods, gtfFile=txdb, filterGenes=FALSE)

Plot the number of genes filtered out and their expression distribution across all sample/gene pairs.

In [ ]:
# FPKM distribution across all sample/gene pairs
plotFPKM(ods) + theme(legend.position = 'bottom')

Let's subset now the object for expressed genes only. Have a look at the output on how many genes/samples (below as `dim`) we ended up for the final analysis.

In [ ]:
# filter object based on the expression status of the genes
ods <- ods[mcols(ods)[,"passedFilter"],]
ods

### Sample co-variation
Usually in RNA-seq data one can see sample co-variation and hence should control for that. In most of the cases one knows the cause of the structure (eg. sex, origin, batch) but sometimes it comes from unkown confounders. For more details [see slides](https://docs.google.com/presentation/d/1a7KZ6FXwVmGqF-FMAnz0QTk07WgzRWAbFhSNrUBspIQ/edit#slide=id.g614aa1ac41_0_33).

In [ ]:
# Make heatmap figure bigger
options(repr.plot.width=6, repr.plot.height=5)

# use normalize=FALSE since the data is not yet corrected
# use columns from the annotation to add labels to the heatmap
plotCountCorHeatmap(ods, colGroups=c("SEX", "ORIGIN"), 
        rowGroups="LAB", normalize=FALSE)

## Detection of expression outlier events

We have loaded the data, did some quality control, and preprocessed (filtered) the data. Now we can start with the **normalization** and **outlier detection**.


<img src="https://github.com/gagneurlab/OUTRIDER/raw/master/vignettes/autoencoder_sketch.png" alt="Using a denoising autoencoder to correct for sample co-variations." width="400"/>

### Fitting the model

We will now use *OUTRIDER* to model the sample co-variation, also called latent space, based on the given gene expression data. Using the correct dimension for the latent space is crucial for achieving the best performance. Finding the right dimension can be done with `findEncodingDim()`. 

In [ ]:
# use BiocParallel to compute in parallel
register(MulticoreParam(workers=2, tasks=10, progressbar=TRUE))

## As this step is heavy to compute we will use a prefitted model.
## If you use your own data please uncomment it and run it.
## It can take 30 minutes to complete depending on the sample/gene size.
ods <- readRDS("outrider/fitted_ods.RDS")

## Find the optimal dimension for the latent space
# ods <- findEncodingDim(ods)
getBestQ(ods)
plotEncDimSearch(ods)

## fit the OUTRIDER model
# ods <- OUTRIDER(ods, q=getBestQ(ods))

Let's see how well the denoising autoencoder fitted the latent space of our data. Ideally, we should not see any cluster anymore and most of the plot should be rather white than red or blue except for the diagonal. Compare the dendogram with the previous heatmap. What do you observe?

In [ ]:
plotCountCorHeatmap(ods, colGroups=c("SEX", "ORIGIN"), 
        rowGroups="LAB", normalize=TRUE)

### Detection of expression outliers

After fitting the model and correcting for sample co-variations, we can now obtain our expression outliers. 
Here, we define an outlier as an event that significantly deviates from the expected Negative-Binomial distribution after controlling for confounders. For details [see slides](https://docs.google.com/presentation/d/1a7KZ6FXwVmGqF-FMAnz0QTk07WgzRWAbFhSNrUBspIQ/edit#slide=id.g64b01c344e_4_158).

<img src="https://github.com/c-mertes/RNAseq-ASHG19/raw/master/src/nb-p-value.PNG" alt="no image" width="250"/>

The two-sided p value is defined as:

$ P(k_{ij}) = 2 \cdot \min\left( \frac{1}{2}, \sum_{k=0}^{k_{ij}} NB(k_{ij}|\mu_{ij}), \theta_j), 1-\sum_{k=0}^{k_{ij}} NB(k_{ij}|\mu_{ij},\theta_j) \right)$,

with a genome-wide significance threshold of $FDR < 0.05$.

Let's check how many expression outliers per sample were found.


In [ ]:
plotAberrantPerSample(ods)

### Z score approach

There are many different ways to define outliers. One common approach is to compute a Z score and then define outliers as $|z| > 3$ or $|z| > 2$. In our [OUTRIDER publication](https://doi.org/10.1016/j.ajhg.2018.10.025), we recommend to use a statistical assesment for defining outliers over the Z score approach.

Despite our recommendation, one can use OUTRIDER to retrieve Z score derived outliers by setting `padjCutoff=1` and `zScoreCutoff=2` or another value of choice.

In [ ]:
plotAberrantPerSample(ods, padjCutoff=1, zScoreCutoff=2, yadjust=c(1.3,1.4))

In order to make the results table more intuitive, we now change the ENSEMBL gene IDs into HGNC symbols. For this we use the [*annotables*](https://github.com/stephenturner/annotables) package.

In [ ]:
# remove versioning of gene IDs and merge with GRCh37 from annotables
geneIDs <- gsub("\\.[0-9]*(_[0-9]*)?.*$", "", rownames(ods))
map <- merge(data.table(ensgene=geneIDs), grch37, sort=FALSE,
        all.x=TRUE)[!duplicated(ensgene),]

# set new gene names only if hgnc symbol is present
if(!"ENSG" %in% colnames(mcols(ods))){
    mcols(ods)$ENSG <- geneIDs
    rownames(ods) <- map[,ifelse(
            is.na(symbol) | symbol == "" | duplicated(symbol), geneIDs, symbol)]
}

Now we can retrieve our expression outliers using the *results* function.

In [ ]:
res <- results(ods)
head(res)
dim(res)

## Finding candidates in a patient, a case study

We know that patient **NA18873** has a rare mitochondrial disease with a complex I deficiency. This means that we are looking for mitochondrial or complex I related genes.

Let's have first a look at the patient's volcano plot. This shows the effect of the event on the x axis (Z score) and the significance on the y axis (nominal p value). This gives an overview of expression status of all genes within a given sample.

In [ ]:
# volcano plot of sample NA18873
plotVolcano(ods, "NA18873", base=TRUE)

We can have then a closer look at the outliers of this specific case in detail by subetting the results table.

In [ ]:
# subset results for this patient
res[sampleID == "NA18873"]

We see 4 outliers in this case. But only one fits the phenotype (e.g. check [GeneCards](https://www.genecards.org) as a reference).

* [CAT](https://www.genecards.org/cgi-bin/carddisp.pl?gene=CAT) 
* [TIMMDC1](https://www.genecards.org/cgi-bin/carddisp.pl?gene=TIMMDC1) 
* [RP11-325F22.2](https://www.genecards.org/cgi-bin/carddisp.pl?gene=ENSG00000237513) 
* [DCTD](https://www.genecards.org/cgi-bin/carddisp.pl?gene=DCTD)


*TIMMDC1* is a known mitochondrial located gene and also part of the complex I. Hence, the best candidate so far. Let's have a closer look on the gene level.

In [ ]:
# make the plot again default size
options(repr.plot.width=8, repr.plot.height=4)

ggarrange(ncol=2,
    plotExpressionRank(ods, "TIMMDC1", norm=FALSE, basePlot=TRUE) + scale_y_log10(lim=c(300,2000)),
    plotExpressionRank(ods, "TIMMDC1", norm=TRUE,  basePlot=TRUE) + scale_y_log10(lim=c(300,2000)))

Wondering why the lowest point in the raw uncorrected counts is not significant? It corresponds to the sample with the lowest *sizeFactor*. The autoencoder uses the *sizeFactor* to correct for different library depths across the dataset and hence the point is gone in the normalized plot. 

To make sure the fit was good, we can look at the quantile-quantile plot.

In [ ]:
# set again the default size for the plot
options(repr.plot.width=4, repr.plot.height=4)

plotQQ(ods, "TIMMDC1")

Interested users can also plot the observed vs the expected gene expression levels to investigate the test statistic with the `plotExpectedVsObservedCounts` function.

## What to do if you see many outliers in a case?

Let's have a look at a second case: **NA11918**.

* How many outliers do you see in this case?
* Are they all disease-related?
* How could you prioritize further to get a handful of candidates only?
* Could a Z score cutoff be helpful?

In [ ]:
# make the plot wider
options(repr.plot.width=8, repr.plot.height=4)

# Volcano plot with significance based on p value or on Z score threshold
ggarrange(ncol=2,
    plotVolcano(ods, "NA11918", main="FDR (<0.05) based volcano plot", 
            base=TRUE, padjCutoff=0.05, zScoreCutoff=0),
    plotVolcano(ods, "NA11918", main="Z score (>3) based volcano plot",
            base=TRUE, padjCutoff=1, zScoreCutoff=3))

In [ ]:
# subset results for this patient (p value based)
res[sampleID == "NA11918"]

And if you want to have the Z score based results table you can extract it as following:

In [ ]:
resZscore <- results(ods, padjCutoff=1, zScoreCutoff=3)
resZscore[sampleID == "NA11918"]

## Expression-Outlier Exercise for later

If you want to test the knowledge you gained in this tutorial, you can try to solve this case by yourself.

What we know:

`HG00103` has an early-onset rare disease and suffers from 
  * microcephaly
  * brain atrophy
  * reduced activities of mitochondrial respiratory complexes I and III

Can you find the disease causing gene for this case?

In [ ]:

# TODO write your code with some documentation. 
# To get you started, some questions below

# * what are the possible candidates
# * what kind of visualization do we have
# * have a detailed look at some of the outliers
# * can we trust the model fit?


Did you find the right candidate? If you want to validate your findings, please have a look at this [paper](https://www.ncbi.nlm.nih.gov/pubmed/26626369).

# Aberrant Splicing Detection in RNA-seq data

A short introduction to splicing can be found in presentation [here](https://docs.google.com/presentation/d/1a7KZ6FXwVmGqF-FMAnz0QTk07WgzRWAbFhSNrUBspIQ/edit?pli=1#slide=id.g64b01c344e_4_394).

## Data import
---

First, we have to load the [*FRASER*](http://github.com/c-mertes/FRASER) package that we will use to run the full aberrant splicing analysis. For efficient data handling and visualization we further load [*data.table*](https://cran.r-project.org/web/packages/data.table/index.html), [*ggplot2*](https://cran.r-project.org/web/packages/ggplot2/index.html), and [*ggpubr*](https://cran.r-project.org/web/packages/ggpubr/index.html).

In [ ]:
library(FRASER)
library(data.table)
library(ggplot2)
library(ggpubr)
register(SerialParam())

We will use the following [dataset](https://i12g-gagneurweb.in.tum.de/public/workshops/RNAseq_ASHG19/input_data/splicing), which is already available in this session.

Due to time reasons, we will go through the standard analysis workflow only with a toy dataset. In the end, we provide a fully preprocessed and fitted model to do the final prioritization.

We assume that the split reads (reads spanning an exon-exon junction) and non-split reads (reads spanning a splice site) are already counted (e.g. with [FRASER](https://github.com/c-mertes/FraseR), [HTSeq](https://htseq.readthedocs.io/en/release_0.11.1/count.html) or [RSubread](https://bioconductor.org/packages/release/bioc/html/Rsubread.html)).

Let's prepare and load the data needed for the aberrant splicing analysis.
* Counts (split reads + non-split reads)
* Sample annotation

In [ ]:
# load and prepare annotation (add needed columns for FRASER)
anno <- fread("annotation.tsv")[,1:6]
anno[, sampleID:=INDIVIDUAL]
## only needed if you start from BAM files and need to count reads first
# anno[, bamFile:="<path/to/sample/bam/file>"]

# get raw counts (remove first column since it is the row index)
junctionCts   <- fread("./splicing/raw_junction_counts.tsv.gz")
spliceSiteCts <- fread("./splicing/raw_site_counts.tsv.gz")

Let's have a look at the count matrix and what extra information we have.

In [ ]:
head(anno)
junctionCts[1:6,1:15]
spliceSiteCts[1:6,1:15]

Now we can put all the data together and create a FRASER dataset object.

In [ ]:
# create FRASER object
fds <- FraserDataSet(colData=anno, junctions=junctionCts, 
        spliceSites=spliceSiteCts)
fds

We will only use a subset to run a fast analysis. Let's take only 3 chromosomes and random 60 samples (keep our cases). Also to speed up the processing, we keep the full data in memory and not on disk using HDF5. We will provide a full fitted model later.

In [ ]:
# randomly subset samples 
set.seed(42)
patient_ids <- c("NA11918", "NA20505", "HG00132")
fds <- fds[
    seqnames(fds) %in% c("chr3", "chr6", "chr19"),
    unique(c(patient_ids, sample(colnames(fds))))[1:60]]
dontWriteHDF5(fds) <- TRUE
fds

## Data preprocessing and QC

As with gene expression analysis, a good quality control of the raw data is crucial. As we will not go into detail here, please refer for some hints to the [gene expression slides](https://docs.google.com/presentation/d/1a7KZ6FXwVmGqF-FMAnz0QTk07WgzRWAbFhSNrUBspIQ/edit?pli=1#slide=id.g64b01c344e_4_51).

### Filtering the data

Before we can filter the data, we have to compute the main splicing metric: the $\psi$-values (Percent Spliced In) and $\theta$-values (splicing efficiency).

In [ ]:
fds <- calculatePSIValues(fds) # take 1-2 minutes


Now we can have some cut-offs to filter down the number of junctions we want to test later on.

Currently, we keep only junctions that support the following:
* At least one sample has 20 reads
* 5% of the samples have at least 1 read

Further, one could filter for:
* At least one sample has to have $|\Delta\psi| > 0.1$

In [ ]:
fds <- filterExpressionAndVariability(fds, minDeltaPsi=0.1, filter=FALSE)

In [ ]:
# make the plot wider
options(repr.plot.width=5, repr.plot.height=4)

plotFilterExpression(fds, bins=100)
plotFilterVariability(fds) + theme(legend.position="none")

After looking at the expression distribution between filtered and unfiltered junctions, we can now subset the dataset. We mainly see that we discarded junctions with overall low expression, which was the intention.

In [ ]:
fds <- fds[mcols(fds, type="j")[,"passed"],]
fds

### Sample co-variation

Since $\psi$ values are ratios within a sample, one might think that there should not be as much correlation structure as observed in gene expression data within the splicing data.

This is not true as we do see strong sample co-variation within many tissues and cohorts. Let's have a look into our data to see if we do have correlation structure or not. To have a better estimate, we use the logit transformed $\psi$ values to compute the correlation. We can annotate our plot with any information provided through the sample annotation.

In [ ]:
# Make heatmap figure bigger
options(repr.plot.width=6, repr.plot.height=5)

plotCountCorHeatmap(fds, type="psi5", logit=TRUE, sampleClustering=NA, 
        annotation_col=c("SEX", "LAB", "ORIGIN"))

## Detection of aberrant splicing events

After preprocessing the raw data and visualizing it, we can start our analysis. Let's start with the first step in aberrant splicing detection: the model fitting.


### Fitting the splicing model

During the fitting procedure, we will normalize the splicing data and correct for confounding effects by using a denoising autoencoder. Using the correct dimension for the latent space is crucial to have the best performance. Finding the right dimension can be done with `optimHyperParams()`.

In [ ]:
# use BiocParallel to compute in parallel
register(MulticoreParam(workers=2, tasks=10, progressbar=TRUE))

## As this step is heavy to compute, we will use a prefitted model.
## If you use your own data please uncomment it and run it.
## It can take 30 minutes to complete depending on the sample/gene size.
fds <- readRDS("splicing/fitted_fds.RDS")

## Find the optimal dimension for the latent space
#for(i in psiType){
#    fds <- optimHyperParams(fds, i)
#    bestQ(fds, i)
#    plotEncDimSearch(fds, i)
#}

## fit the FRASER model
#fds <- FRASER(fds)

Let's see how well the correction worked by looking at the correlation heatmap again.

In [ ]:
plotCountCorHeatmap(fds, type="psi5", normalized=TRUE, logit=TRUE, topN=15000, 
        annotation_col=c("SEX", "LAB", "ORIGIN"), sampleClustering=NA)

### Calling splicing outliers

For aberrant splicing detection, we assume a beta binomial distribution and call outliers based on the significance level. But again the user can choose between a p value cutoff, a Z score cutoff or both. We do recommend to use a pure p value cutoff.

Let's have a look how many splicing outliers we detect by sample.

In [ ]:
plotAberrantPerSample(fds)

Again, before we extract the results let's add the interpretable HGNC symbols. *FRASER* comes already with an annotation function. The function uses a user provided TxDb object (defaults to *UCSC.hg19*) to overlap the genomic ranges with the known HGNC symbols. 

In [ ]:
# annotate
fds <- annotateRangesWithTxDb(fds)

# retrive results
register(SerialParam())
res <- as.data.table(results(fds))
res

As FRASER tests $\psi_5$, $\psi_3$ and $\theta$, it can reports multiple hits for the same event. This is helpful later when interprating the results. But of a first overview this can be overwhelming. Hence, we suggest to aggregrate the results first at the gene level before going into details.

In [ ]:
# convert back to GRange object
resAsGR <- makeGRangesFromDataFrame(res, keep.extra.columns = TRUE)

# group results by genes/sample
results_by_genes <- as.data.table(resultsByGenes(resAsGR))
results_by_genes

## Finding splicing candidates in a patient

We worked already with the following patient: **NA11918**, in whom we found downregulation of **TIMMDC1**.

Let's see if we also got some splicing candidates for the same sample.

In [ ]:
# Make the plotting area smaller again
options(repr.plot.width=4, repr.plot.height=4)

plotVolcano(fds, type="psi5", "NA11918")

Which are the splicing events in detail? Do they fit with our previous candidate? 

In [ ]:
res[sampleID == "NA11918"]

Since we do detect the same gene again (found significantly down regulated), let's have a look at the junction level.

In [ ]:
plotExpression(fds, type="psi5", result=res[sampleID == "NA11918" & hgncSymbol == "TIMMDC1"][1])

Finding the same gene through two different approaches increases the confidence in the event. Further, it gives clues on the mechanism of the event (eg. NMD through alternative isoforms). But further investigations of causal variants are needed.

When it comes to a diagnosis, we want to make sure that our call is correct. Hence, we want to look at many metric to boost our confident into the call. Let's make a publication ready figure (except the sashimi plot).

In [ ]:
# Make the plotting area fit 4 panels
options(repr.plot.width=9, repr.plot.height=4)

res2plot <- res[sampleID == "NA11918" & hgncSymbol == "TIMMDC1"][1,]
ggarrange(ncol=2,
    plotVolcano(fds, type="psi5", "NA11918"),
    plotExpression(fds, result=res2plot),
    plotQQ(fds, result=res2plot),
    plotExpectedVsObservedPsi(fds, result=res2plot)
)

In [ ]:
# Make the plotting area smaller again
options(repr.plot.width=4, repr.plot.height=4)

## Aberrant Splicing Exercise for later

If you want to test the knowledge you gained in this tutorial, you can try to solve this case by yourself.

What we know:

`NA20505` has an early-onset rare disease and suffers from 
  * myopathic facies
  * cardiac arrhythmias

Can you find the disease causing gene for this case?

You can check this [paper](https://www.biorxiv.org/content/10.1101/2019.12.18.866830v1) to overlap it with your findings.

In [ ]:

# TODO write your code with some documentation. 
# To get you started, some questions below

# * what are the possible candidates
# * what kind of visualization do we have
# * have a detailed look at some of the outliers
# * can we trust the model fit?
# * do we see an overlap with gene expression?


# Mono Allelic Expression

For a short introduction to MAE please refer to the [slides](https://docs.google.com/presentation/d/1a7KZ6FXwVmGqF-FMAnz0QTk07WgzRWAbFhSNrUBspIQ/edit?pli=1#slide=id.g64b01c344e_4_388).

## Load the allelic counts
---

For the MAE analysis we will use the *tMAE* package.


In [ ]:
library(ggplot2)
library(data.table)
library(tMAE)

Now we can load the allelic counts. For this tutorial we only use 2 samples and a subset of the variants. 


In [ ]:
# allelic counts generated using ASEReadCounter
allelicCountsFile <- 'https://i12g-gagneurweb.in.tum.de/public/workshops/RNAseq_ASHG19/input_data/mae/allelic_counts.tsv.gz'
allelicCounts <- fread(allelicCountsFile)

# print data
allelicCounts[1:4,]
dim(allelicCounts)

Check the existing samples in the table.

In [ ]:
print('IDs in table')
unique(allelicCounts$MAE_ID)

Plot counts of alternative vs reference allele. Which ones would you say are mono-allelically expressed?

In [ ]:
ggplot(allelicCounts, aes(refCount+1, altCount+1)) + geom_point() +
    geom_abline(slope=1, intercept=0) + 
    scale_y_log10() + scale_x_log10() + theme_bw()


## Run MAE test
---
Now we are ready to run the negative binomial test using the `DESeq4MAE` function from the *tMAE* package

In [ ]:
resMAE <- DESeq4MAE(allelicCounts, minCoverage = 10)
head(resMAE)

### Determine the number of mono-allelic events

We define a variant to be mono-allelically expressed if its p-adjusted < 0.05. Determine how many mono-allelic events there are per sample.




In [ ]:
# Add significant column
resMAE[, signif := padj < 0.05]

# Get number of cases
print('MAE significant variants')
resMAE[signif == TRUE, .N, by = MAE_ID]

In the rare disease setting, variants that are mono-allelically expressed for the altervative are more interesting. We define them as having a ratio of the alternative $\ge 0.8$ and an $FDR < 0.05$. 

Determine how many mono-allelic events for the **alternative** allele there are per sample.

In [ ]:
# Add column for significant mono-allelic expression of the alternative
resMAE[, signif_ALT := signif == TRUE & altRatio >= 0.8]

# Get number of cases
print('MAE for the alternative significant variants')
resMAE[signif_ALT == TRUE, .N, by = MAE_ID]

## Find rare variants and integrate them to results
---
Add minor allele frequencies from [gnomAD](https://gnomad.broadinstitute.org/). The following code snippet requires the gnomAD libraries to be downloaded which are ~5GB each as they contain the minor allele frequencies of the whole genome. Therefore, we will not execute it here, but simply read the results object with annotated allele frequencies.

In [ ]:
## If your data is based on assembly hg37
# library(MafDb.gnomAD.r2.1.hs37d5)
# mafdb <- MafDb.gnomAD.r2.1.hs37d5

## If your data is based on assembly hg38
# library(MafDb.gnomAD.r2.1.GRCh38)
# mafdb <- MafDb.gnomAD.r2.1.GRCh38

## convert results table into GRanges object
# rng <- GRanges(seqnames = data$contig, 
#               ranges = IRanges(start = data$position, end = data$position), 
#               strand = '*')
# resMAE$gnomadAF <- gscores(mafdb, rng)$AF


Read the annotated results. How many rare variants are there?

In [ ]:
# Merge results with annotation object
resAnnot <- fread('https://i12g-gagneurweb.in.tum.de/public/workshops/RNAseq_ASHG19/input_data/mae/mae_annotated_results.tsv.gz')

resAnnot[, rare := (gnomadAF <= 0.01 | is.na(gnomadAF))]
print('Number of rare events in total (including non significant variants)')
resAnnot[rare == TRUE, .N]


Which are the rare mono-allelic events of these 2 samples?

In [ ]:
resAnnot[signif_ALT == TRUE & rare == TRUE]

Visualize them using the `plotAllelicCounts` function from the *tMAE* pkg.

In [ ]:
sample1 <- 'HG00106'
plotAllelicCounts(resAnnot[MAE_ID == sample1], 
        rare_column = 'rare', title = sample1) +
    theme(legend.position="bottom")


In [ ]:
sample2 <- 'HG00111'
plotAllelicCounts(resAnnot[MAE_ID == sample2], 
        rare_column = 'rare', title = sample2) + 
    theme(legend.position="bottom")

After running the test and adding minor allele frequencies for all the 100 samples in the data set, we obtained the following results for each filter:

<img src="https://github.com/c-mertes/RNAseq-ASHG19/raw/master/src/cascade_plot.png" alt="no image" width="400"/>

For further validation, obtain the gene and consequence of the variant from the annotated VCF file (more info on next section).

# Gene prioritization


**It is not uncommon to have more than one candidate outlier gene when looking at expression, splicing or allele specific events. It is useful to know that there are ways to prioritize those genes.
Here, we will see how coming back to phenotype and genotype data can help reduce the list of candidates in a case-informed way.** 

Because of time constraint, we will focus on an example with genes obtained from expression outliers. In real life, you can apply the same strategy to other types of candidates, such as splicing or MAE outliers. 




## Set up working session
---

First a few packages need to be loaded. \\
(1) [VariantAnnotation](https://bioconductor.org/packages/release/bioc/html/VariantAnnotation.html): annotate variants, compute amino acid coding changes, predict coding outcomes. \\
(2) [TVTB](https://bioconductor.org/packages/release/bioc/html/TVTB.html) (The VCF ToolBox): filter, summarize and visualize genetic variation data stored in VCF files. \\
(3) [annotables](https://github.com/stephenturner/annotables): annotating/converting Gene IDs. \\
(4) [tidyverse](https://www.tidyverse.org/): collection of packages designed for data science \\
(5) [data.table](https://cran.r-project.org/web/packages/data.table/vignettes/datatable-intro.html): data manipulation operations 

In [ ]:
library(VariantAnnotation)
library(TVTB)
library(annotables)
library(ensemblVEP)
library(tidyverse)
library(data.table)



## Case 1
This boy was the third child of healthy non-consanguineous French parents. Pregnancy and delivery were uneventful. Early psychomotor development was normal. However, speech development was delayed, acquiring language at the age of 4. At 11, he began to experience psychomotor regression and progressive visual loss. At current (by the time of the publication) age of 47, he has severe walking difficulties, blindness, abnormal behaviour (easily frightened, sometimes aggressive) and spontaneous speech.

In our dataset, this sample is named **NA11918**.

### Get list of candidate genes from OUTRIDER
We download the list of outlier gene that was generated using OUTRIDER in an earlier session.

In [ ]:
# Download list of candidates
results_link="https://i12g-gagneurweb.in.tum.de/public/workshops/RNAseq_ASHG19/input_data/outrider/results_pvalue.tsv.gz"
outrider_results_genes=fread(results_link)

# Filter for the case of interest
outrider_results_genes=outrider_results_genes %>%
        filter(sampleID=="NA11918") %>%
        select(geneID) %>%
        unlist

# Take a look at what the genes list look like
head(outrider_results_genes)

# Get the number of candidate genes for that case
print("The total number of candidate genes within this case:")
length(outrider_results_genes)


You can see that the gene names are not totally in an Ensembl format. The number you see corresponds to a version of the annotation.
The gene names need to be reformated so that we keep only the official gene name in our analyses.


We also see that we have 22 candidate genes.

In [ ]:
# Modify gene format
outrider_results_genes=str_extract(outrider_results_genes, "ENSG[0-9]+")


# Look at the changes
head(outrider_results_genes)


Now that the gene name corresponds to the official Ensembl symbols, it is possible to get more information on those. 
In particular we can easily obtain their symbol and description using the package ` annotable `.


In [ ]:
# Query annotable Grch37 object and keep only genes that are in the list of outliers
candidate_genes=grch37 %>% filter(ensgene %in% outrider_results_genes)

# Look at the results
head(candidate_genes)

We now have the list of candidate genes and more information on what they are. We are ready to filter this list to keep only the genes that are linked to the symptoms of the case of interest.

In our case, the symptoms are:
* Developmental regression ([HP:0002376](https://hpo.jax.org/app/browse/term/HP:0002376))
* Ataxia([HP:0001251](https://hpo.jax.org/app/browse/term/HP:0001251))
* Ophthalmoplegia ([HP:0000602](https://hpo.jax.org/app/browse/term/HP:0000602))
* Visual impairment ([HP:0000505](https://hpo.jax.org/app/browse/term/HP:0000505))


As you can see, there is an ID beside each symptom. Those are HPO term IDs, as found in the Human Phenotype Ontology [database](https://hpo.jax.org/app/).


### Annotate candidate genes with HPO terms
#### Read in sample phenotype metadata


In [ ]:
# We define the list of HPO terms for the case
sample_HPO=c("HP:0002376","HP:0001251", "HP:0000602","HP:0000505")

sample_HPO

#### Load gene to HPO ID file from HPO database 
The Human Phenotype Ontology gives access to a very useful file, showing the link between symptoms and genes. This file is updated regularly so we recommend to **download the latest version** whenever possible.

You can download this file [here](http://compbio.charite.de/jenkins/job/hpo.annotations.monthly/lastStableBuild/artifact/annotation/ALL_SOURCES_FREQUENT_FEATURES_phenotype_to_genes.txt).

In [ ]:

# Get the file URL
hpo_annotations_url ="http://compbio.charite.de/jenkins/job/hpo.annotations.monthly/lastStableBuild/artifact/annotation/ALL_SOURCES_FREQUENT_FEATURES_phenotype_to_genes.txt"

# Read the file
gene_hpo <-fread(hpo_annotations_url, skip=1)

# Select only the columns of interest (Gene and HPO term)
gene_hpo <- gene_hpo[, c(4, 1)]
colnames(gene_hpo) <- c("Gene", "Term")

# Look at the results
head(gene_hpo)

#### Get a subset of genes that could match the symptoms
Now that we have downloaded that file, we want to keep only the genes that are phenotypically relevant to the case.

1. Filter for HPO terms corresponding to the case
2. Obtain the Ensembl IDs for the genes that are associated to those HPO terms


In [ ]:
# Get list of genes corresponding to the case symptoms.
genes_hpo_case=gene_hpo %>% 
        filter(Term %in% sample_HPO) %>% 
        left_join(grch37,by=c("Gene"="symbol"))
genes_hpo_case=genes_hpo_case %>% 
        select(ensgene) %>% 
        unique %>% 
        unlist
head(genes_hpo_case)

`genes_hpo_case` contains the Ensembl IDs of genes associated with HPO terms for the case.

You can quicly check how many genes are currently linked with the symptoms of your case of interest.

In [ ]:
# Get number of genes linked to case symptoms
length(genes_hpo_case)

There are 1,519 genes that could match some of the patient's symtoms.



---



#### Subset the list of genes to outlier candidates 
Here the assumption is that if there is an expression perturbation on the causal gene, this gene is somehow linked to some of the symptoms of the patient.\
So we want to **filter** the list of genes somehow linked to the patients symptoms (listed in `genes_hpo_case`) for the one obtained as candidates in previous work.


In [ ]:
# Filter outlier genes for genes linked to the phenotype.
candidate_genes.hpo=candidate_genes %>% filter(ensgene %in% genes_hpo_case)

# Get the number of genes left
cat("The number of outlier genes left after filtering associated with the disease:")
length(candidate_genes.hpo$ensgene)

# Take a look at the results
candidate_genes.hpo

As you can see, the list of potential genes is now much reduced. We dropped from 22 candidates to 2 genes, [*MCOLN1*](https://www.genecards.org/cgi-bin/carddisp.pl?gene=MCOLN1) and [*DNMT3A*](https://www.genecards.org/cgi-bin/carddisp.pl?gene=DNMT3A).


---

---

### Annotate with variant information

We can go further and annotate our candidate with the variant information for the case.

To do so, we need to transform the candidate gene list into the right format so that we can use the genes when reading in the [VCF](https://www.internationalgenome.org/wiki/Analysis/Variant%20Call%20Format/vcf-variant-call-format-version-40/) file in R.

#### Build a GRanges object with the genes obtain in previous steps
We use R packages that have been developped to handle and filter VCF files. 
Here, we want to upload only the part of the VCF file that is in regions of interest (i.e., our candidate genes). We can do this by giving as an input an object of type [GRanges](https://web.mit.edu/~r/current/arch/i386_linux26/lib/R/library/GenomicRanges/html/GRanges-class.html).


The next steps are transforming our list of candidate genes to the right format so that we can proceed with the analysis.

1. Change the start and end position to include potential regulatory regions (+/- 1kb here).
Note that you can customize this distance or use the gene coordinates.


In [ ]:
# Extending the start/end coordinates to 1kb around the gene of interest 
candidate_genes=candidate_genes.hpo %>% 
        mutate(new_start=pmin(start,end)-1e3, new_end=pmax(start,end)+1e3)

# Take a look at the results
head(candidate_genes)

2. Transform data frame into GRanges object

In [ ]:
# Create a GRanges Object
candidate_genes.gr=makeGRangesFromDataFrame(
        candidate_genes, ignore.strand=TRUE,
        start.field="new_start",end.field="new_end")

# Add gene names to GRanges object
names(candidate_genes.gr)=candidate_genes$ensgene

candidate_genes.gr

#### Load the vcf file
Now that we have the coordinates of the candidate genes in the right format, we can process the VCF file.

For the purpose of this workshop, the VCF file needs to be annotate with [Variant Effect Predictor](https://uswest.ensembl.org/info/docs/tools/vep/index.html) (VEP), which determines the effect of the variants on genes.

Here, we take a subset of the VCF file from the 1000 Genomes Project in which we have injected variants of interest.


In [ ]:
# VEP annotated VCF file
vcfFile <- "./variants/1000G_subset_exome.vep.vcf.gz"

# index the VCF file
indexVcf(vcfFile)

# create a reference 
vcfFile <- TabixFile(vcfFile)
vcfFile


#### Read in VCF file of the sample of interest
Here, only the sample of interest and variants overlapping the genes of interest will be looked at.


In [ ]:
# Create parameter file to read only the data for the case and around candidate genes
params=ScanVcfParam(samples="NA11918", which=candidate_genes.gr)

# Read in VCF file filtering data using the parameter file we just created
vcf_rng <- readVcf(vcfFile, "hg19", params)
head(rowRanges(vcf_rng), 3)

#### Filter for Heterozygous or Homozygous Alt and genes of interest
The VCF contains genotypes for sites that are homozygous for the reference allele. As we are not interested in those we filter them out.
We also make sure that we are looking only at the variants that are annotated for the genes of interest and not genes nearby.

In [ ]:
candidate_genes

In [ ]:
# Create a filter on variants het or homozygous alt
Hetfilt <- FilterRules(list(HetorHomAlt = 
        function(x) geno(x)$GT %in% c("0|1", "1|1", "1|0")))

# Create a filter to keep only candidate gene annotations
GeneFilt<-VcfVepRules(exprs = list(Cand_genes = 
        bquote(SYMBOL %in% .(candidate_genes$symbol) )))                            

# Combine those filters
combinedPreFilters <- VcfFilterRules(
    Hetfilt, 
    GeneFilt)
                            
# Apply them on the vcf
vcf_het_cand <- subsetByFilter(vcf_rng, combinedPreFilters)

rowRanges(vcf_het_cand)

#### Get consequence field
In order to see the potential features we can filter on, let's take a look at the consequence field of the VCF.

In [ ]:
# Parse the consequence field of the VCF
csq <- parseCSQToGRanges(x=vcf_het_cand, VCFRowID=rownames(vcf_het_cand))
csq[, c("Consequence", "SYMBOL", "BIOTYPE", "gnomAD_AF", "CADD_PHRED")]



#### Define a set of filters

We can create a set of filters that we want to apply on our data, such as distance to transcription start site, allele frequency, [CADD score](https://cadd.gs.washington.edu/), [VEP consequences](https://uswest.ensembl.org/info/genome/variation/prediction/predicted_data.html), etc. You can customize this and add as many filters as you want.

In [ ]:

# Filter on distance to the gene
vepDistFilter <- VcfVepRules(exprs=list(Distance=expression(DISTANCE <= 1000)))

# Filter on allele frequency
# Here we allow NAs because some variants are uniq to the individual and hence
# not listed in any public database (eg. gnomAD)
vepMAFFilter<- VcfVepRules(exprs = list(MAF = 
        expression(as.numeric(gnomAD_AF) <= 0.01 || gnomAD_AF == "NA")))

# Filter on CADD score
vepCADDFilter <- VcfVepRules(exprs = list(CADD=expression(CADD_PHRED >= 20)))

# Filter on consequences
highImpactVariant<-VcfVepRules(exprs=list(BigImpact=
        expression(grepl(x=Consequence, pattern=paste(collapse="|", c(
                "splice_acceptor_variant", "splice_donor_variant", 
                "stop_gained", "stop_lost","frameshift_variant"))))))

regulatoryVariant<-VcfVepRules(exprs=list(RegVar=
        expression(grepl(x=Consequence, pattern=paste(collapse="|", c(
                "5_prime_UTR_variant", "3_prime_UTR_variant", "intron_variant",
                "NMD_transcript_variant", "upstream_gene_variant", 
                "downstream_gene_variant"))))))

combinedFilters <- VcfFilterRules(
  vepDistFilter,
  vepMAFFilter,
  vepCADDFilter, 
  highImpactVariant,
  regulatoryVariant)

You can play and add or remove filters

In [ ]:
# Look at active filters
active(combinedFilters)

We focus only on **rare** variants

In [ ]:
active(combinedFilters)["BigImpact"] <- FALSE
active(combinedFilters)["RegVar"] <- FALSE
active(combinedFilters)["Distance"] <- FALSE
active(combinedFilters)["CADD"] <- FALSE

active(combinedFilters)["MAF"] <- TRUE

active(combinedFilters)

#### Apply set of filters

In [ ]:
# subset VCF with active filters
vcf_filt <- subsetByFilter(vcf_het_cand, combinedFilters)

csq_filt <- ensemblVEP::parseCSQToGRanges(x = vcf_filt)

unique(csq_filt[,c("Consequence", "SYMBOL", "BIOTYPE", "gnomAD_AF","CADD_PHRED")])

There are only two rare variants in our example. They are both in the same gene, *MCOLN1* which was previously involved in a [disorder](https://www.omim.org/entry/252650#title) linked to ophthalmologic abnormalities, difficulty to speak, slow development. This gene looks like a very good candidate for our case. The presence of those two mutations, one leading to an intron retention and the other one leading to a stop codon are good candidates.

In order to see how many variants are left after each filter, you can use the following command. Please note that filters that are not active will have no effect on the initial number of variants.



In [ ]:
# Look at number of variants left for each set of filters
summary(evalSeparately(expr=combinedFilters, envir=vcf_het_cand))




---



---



## Playground Case 2
You can try to prioritize other genes by looking at this case **HG00132**.


~12 yo Hispanic female with global developmental delay after normal development until age 18 months, then loss of milestones, head control, and speech; tremors at 21 months; and seizures at 22 months.  Occasional myoclonus, not correlative to EEG.  Gastrostomy fed.  Scoliosis. Brother similiarly affected.


The HPO terms for this case are: HP:0001250,HP:0001298,HP:0002015,HP:0002650,HP:0001263,HP:0002376,HP:0002421,HP:0002371,HP:0001337,HP:0001336,HP:0001260,HP:0003676

The results file for splicing outliers is [here](https://i12g-gagneurweb.in.tum.de/public/workshops/RNAseq_ASHG19/input_data/splicing/results_pvalue.tsv.gz)


The vcf containg variant infromation for HG00132 is [here](https://i12g-gagneurweb.in.tum.de/public/workshops/RNAseq_ASHG19/input_data/variants/1000G_subset_exome.vep.vcf.gz)


This case was solved in this [paper](https://www.nature.com/articles/s41591-019-0457-8)